In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256),
                    nn.ReLU(),
                    nn.Linear(256,10))

# 在[0,1)之间随机抽取(2,20)的矩阵
X = torch.rand(2,20)
net(X)

tensor([[-0.1337, -0.0974,  0.1162,  0.0413, -0.0723,  0.0611, -0.1146, -0.1125,
         -0.0758,  0.1083],
        [-0.0627, -0.2440,  0.0999,  0.0108,  0.1183, -0.0310, -0.0207, -0.0878,
         -0.0574,  0.1163]], grad_fn=<AddmmBackward0>)

In [2]:
# 任意一个神经网络或者一个层都应该是module的子类
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)

    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[-0.0667,  0.0926, -0.0072, -0.1566,  0.0473, -0.2431,  0.0336,  0.1167,
          0.0484, -0.0826],
        [-0.0777, -0.0487,  0.0197,  0.0008,  0.0903, -0.2826, -0.0420,  0.1677,
          0.0950, -0.1425]], grad_fn=<AddmmBackward0>)

In [5]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20,256),
                   nn.ReLU(),
                   nn.Linear(256,10))
net(X)

tensor([[ 0.0320,  0.1798,  0.0007,  0.0343, -0.0813, -0.0659,  0.0861, -0.1683,
         -0.0615,  0.0100],
        [ 0.0366,  0.1014,  0.1920, -0.0056, -0.0520,  0.0228, -0.0336, -0.1652,
          0.1278,  0.0421]], grad_fn=<AddmmBackward0>)

In [6]:
# 在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)

    def forward(self,X):
        X = self.linear(X)
        X = F.relu(torch.mm(X,self.rand_weight)+1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.3041, grad_fn=<SumBackward0>)

In [7]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                 nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)

    def forward(self,X):
       return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(0.0710, grad_fn=<SumBackward0>)